In [1]:
import pandas as pd

wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[["alcohol", "sugar", "pH"]]
target = wine["class"].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [6]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00507593, 0.00455308, 0.00472283, 0.0044229 , 0.00414491]), 'score_time': array([0.00078702, 0.00065494, 0.00090122, 0.00051212, 0.00050402]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np

print(np.mean(scores["test_score"]))

0.855300214703487


In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores["test_score"]))

0.855300214703487


In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores["test_score"]))

0.855300214703487


In [13]:
from sklearn.model_selection import GridSearchCV

params = {"min_impurity_decrease" : [0.001, 0.002, 0.003, 0.004, 0.005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004,
                                                   0.005]})

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)
print(gs.cv_results_["mean_test_score"])

0.8724263998460651
{'min_impurity_decrease': 0.001}
[0.86165044 0.85761105 0.85607111 0.85414803 0.85318557]


In [15]:
best_index = np.argmax(gs.cv_results_["mean_test_score"])
print(gs.cv_results_["params"][best_index])

{'min_impurity_decrease': 0.001}


In [16]:
params = { "min_impurity_decrease" : np.arange(0.0001, 0.001, 0.0001),
            "max_depth" : range(5, 20, 1), 
            "min_samples_split" : range(2, 100, 10)}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_["mean_test_score"]))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [21]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([5, 7, 1, 9, 2, 1, 4, 9, 1, 1])

In [22]:
np.unique(rgen.rvs(1000), return_counts=True)


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110,  93, 106,  97, 108,  96, 102,  92, 103,  93]))

In [23]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.74330726, 0.56899184, 0.64080254, 0.2998925 , 0.90669   ,
       0.95232308, 0.9337144 , 0.94128469, 0.12402033, 0.92635796])

In [24]:
params = {"min_impurity_decrease" : uniform(0.0001, 0.001),
          "max_depth" : randint(20, 50),
          "min_samples_split" : randint(2, 25), 
          "min_samples_leaf" : randint(1, 25)}

In [25]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x2867bdac0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x172c4bd30>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x2867bd5b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x17f9568b0>},
                   random_state=42)

In [26]:
print(gs.best_params_)
print(np.max(gs.cv_results_["mean_test_score"]))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [27]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
